<a href="https://colab.research.google.com/github/xtevensab007/Modelos_1/blob/main/fase-1/02_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import warnings
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

# Carga de datos
data_train = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQHU9r-osSfdm1EQ1qT4SqM13kdFN5ELQStnSx-n4jOzSr4jxrg-YMG4bCe9W5Kkw1qelHxSKrfZlMi/pub?gid=1834913949&single=true&output=csv')
data_test = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSOz7KZRcRa_l7f4QF5wk5W5LXvAZZBMgr5hdVGt2d_Pxm8n4fqnPQ4n6dSLv0H2nkNx19SKUzGZhX4/pub?gid=854072752&single=true&output=csv')


/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [ ]:
# nombres de las columnas test
column_names = data_test.columns

print(column_names)


Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew'],
      dtype='object')


# Modelos
###Modelo con Random Forest

Para este primer acercamiento se analizará primordialmente  las variables de ('budget', 'popularity', 'runtime') puesto se les considera los factores mas influyentes.

In [ ]:
# columnas
columnas = ['budget', 'popularity', 'runtime']
X_train = data_train[columnas]
y_train = data_train['revenue']

X_test = data_test[columnas]

# Filtrar valores nulos
X_train = X_train.fillna(0)
y_train = y_train.fillna(0)
X_test = X_test.fillna(0)

# Limpiar y transformar las columnas con formato incorrecto
for columna in columnas:
    X_train[columna] = pd.to_numeric(X_train[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')
    X_test[columna] = pd.to_numeric(X_test[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')

# Eliminar valores nulos en X_train y y_train
filas_nulas_entrenamiento = X_train[X_train.isnull().any(axis=1) | y_train.isnull()]
X_train = X_train.drop(filas_nulas_entrenamiento.index)
y_train = y_train.drop(filas_nulas_entrenamiento.index)

# Dividir datos de entrenamiento
X_train_entrenamiento, X_train_evaluacion, y_train_entrenamiento, y_train_evaluacion = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Crear modelo de bosque aleatorio
modelo = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)

# Entrenar el modelo
modelo.fit(X_train_entrenamiento, y_train_entrenamiento)

# Realizar predicciones en el conjunto de evaluación
predicciones_evaluacion = modelo.predict(X_train_evaluacion)

# Error Cuadrático Medio (MSE)
mse = mean_squared_error(y_train_evaluacion, predicciones_evaluacion)
print(f'Error Cuadrático Medio en el conjunto de evaluación: {mse}')

# Error Porcentual Absoluto Medio (MAPE)
mape = mean_absolute_error(y_train_evaluacion, predicciones_evaluacion) / y_train_evaluacion.mean() * 100
print(f'Error Porcentual Absoluto Medio en el conjunto de evaluación: {mape}%')


Error Cuadrático Medio en el conjunto de evaluación: 1229573966511585.0
Error Porcentual Absoluto Medio en el conjunto de evaluación: 83.00649221608263%


Es evidente que el error es demasiado elevado, con locual se concluye que a pesar de que parece que estas variables son las mas influyentes a primera vista, no son suficientemente descriptivas en un ambito general.

A partir de de este entendimiento, se integrarán mas columnas al modelo, esperando que mejore la calidad de la predicción


In [ ]:

# columnas
columnas_seleccionadas = ['budget', 'genres', 'original_language', 'popularity', 'production_companies',
                           'production_countries', 'release_date', 'runtime', 'spoken_languages', 'cast', 'revenue']

data_train = data_train[columnas_seleccionadas]
data_test = data_test[columnas_seleccionadas[:-1]]  # Excluir 'revenue' de las columnas de prueba

# valores nulos
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

# Limpiar
columnas_numericas = ['budget', 'popularity', 'runtime']
for columna in columnas_numericas:
    data_train[columna] = pd.to_numeric(data_train[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')
    data_test[columna] = pd.to_numeric(data_test[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')

# columnas categóricas
columnas_categoricas = ['genres', 'original_language', 'production_companies', 'production_countries', 'spoken_languages', 'cast']
data_train[columnas_categoricas] = data_train[columnas_categoricas].astype(str)
data_test[columnas_categoricas] = data_test[columnas_categoricas].astype(str)

# Codificación one-hot
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Aplicar codificación one-hot
data_train_encoded = pd.DataFrame(encoder.fit_transform(data_train[columnas_categoricas]))
data_train_encoded.columns = data_train_encoded.columns.astype(str)

data_test_encoded = pd.DataFrame(encoder.transform(data_test[columnas_categoricas]))
data_test_encoded.columns = data_test_encoded.columns.astype(str)

# Concatenar las columnas
data_train_final = pd.concat([data_train[columnas_numericas], data_train_encoded], axis=1)
data_test_final = pd.concat([data_test[columnas_numericas], data_test_encoded], axis=1)

# Dividir datos
X_train_entrenamiento, X_train_evaluacion, y_train_entrenamiento, y_train_evaluacion = train_test_split(data_train_final, data_train['revenue'], test_size=0.2, random_state=42)

# Crear modelo de Random Forest
modelo = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)

# Entrenar modelo
modelo.fit(X_train_entrenamiento, y_train_entrenamiento)

# Realizar predicciones
predicciones_evaluacion = modelo.predict(X_train_evaluacion)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Calcular el Error Cuadrático Medio (MSE) y Error Porcentual Absoluto Medio (MAPE)
mse = mean_squared_error(y_train_evaluacion, predicciones_evaluacion)
print(f'Error Cuadrático Medio en el conjunto de evaluación: {mse}')

mape = mean_absolute_error(y_train_evaluacion, predicciones_evaluacion) / y_train_evaluacion.mean() * 100
print(f'Error Porcentual Absoluto Medio en el conjunto de evaluación: {mape}%')


Error Cuadrático Medio en el conjunto de evaluación: 6616851962288012.0
Error Porcentual Absoluto Medio en el conjunto de evaluación: 61.45531854162958%


Se puede apreciar un menor porcentaje de error, sin embargo la difecencia es minima y este sigue siendo relativamente alto, a pesar de la inclusión de nuevas variables que parecieran ser mas significativas e influyentes en el valor del dinero recaudado por estas peliculas

###modelo con SVM

Se realizará una prueba con el modelo SVM con la intencion de comparar resultados


In [ ]:
#  valores nulos
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

# Limpiar
columnas_numericas = ['budget', 'popularity', 'runtime']
for columna in columnas_numericas:
    data_train[columna] = pd.to_numeric(data_train[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')
    data_test[columna] = pd.to_numeric(data_test[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')

# Convertir categóricas
columnas_categoricas = ['genres', 'original_language', 'production_companies', 'production_countries', 'spoken_languages', 'cast']
data_train[columnas_categoricas] = data_train[columnas_categoricas].astype(str)
data_test[columnas_categoricas] = data_test[columnas_categoricas].astype(str)

# Codificación one-hot
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Aplicar codificación one-hot
data_train_encoded = pd.DataFrame(encoder.fit_transform(data_train[columnas_categoricas]))
data_train_encoded.columns = data_train_encoded.columns.astype(str)

data_test_encoded = pd.DataFrame(encoder.transform(data_test[columnas_categoricas]))
data_test_encoded.columns = data_test_encoded.columns.astype(str)

# Concatenar las columnas
data_train_final = pd.concat([data_train[columnas_numericas], data_train_encoded], axis=1)
data_test_final = pd.concat([data_test[columnas_numericas], data_test_encoded], axis=1)

# Dividir datos
X_train_entrenamiento, X_train_evaluacion, y_train_entrenamiento, y_train_evaluacion = train_test_split(data_train_final, data_train['revenue'], test_size=0.2, random_state=42)

# Crear modelo de SVM con un pipeline que incluye imputación de valores nulos
modelo = make_pipeline(SimpleImputer(strategy='mean'), SVR())

# Entrenar el modelo
modelo.fit(X_train_entrenamiento, y_train_entrenamiento)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('simpleimputer', SimpleImputer()), ('svr', SVR())])

In [ ]:
# Realizar predicciones
predicciones_evaluacion = modelo.predict(X_train_evaluacion)

# Error Cuadrático Medio (MSE) y Error Porcentual Absoluto Medio (MAPE)
mse = mean_squared_error(y_train_evaluacion, predicciones_evaluacion)
mape = mean_absolute_error(y_train_evaluacion, predicciones_evaluacion) / y_train_evaluacion.mean() * 100

print(f'Error Cuadrático Medio en el conjunto de evaluación: {mse}')
print(f'Error Porcentual Absoluto Medio en el conjunto de evaluación: {mape:.2f}%')


Error Cuadrático Medio en el conjunto de evaluación: 1.9866483239291616e+16
Error Porcentual Absoluto Medio en el conjunto de evaluación: 93.01%


## conclusiones

De la presente evaluación es posible concluir que el modelo de Support Vector Machine (SVM) con el conjunto de datos proporcionado parece tener un rendimiento significativamente inferior en compración al modelo de Random Forest. Existe un multiplicidad de factores que pueden contribuir a esta discrepancia, uno de estos radica en la sensibilidad a la escala de las características en SVM, del mismo modo, la necesidad de ajustar adecuadamente los hiperparámetros y el manejo de valores nulos. Podríamos afirmar entonces que en comparación con Random Forest, que puede ser más robusto y menos sensible a la escala, es recomendable centrarse en ajustar y mejorar el modelo de Random Forest, considerando entonces la exploración de diferentes configuraciones de hiperparámetros y asegurándose de que los datos se preparen adecuadamente previo a entrenar el modelo.
